
<h1 id="Electrical-Power-Generation-1">Electrical Power Generation 1<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power_1.html#Electrical-Power-Generation-1">¶</a></h1><h2 id="Objective-and-Prerequisites">Objective and Prerequisites<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power_1.html#Objective-and-Prerequisites">¶</a></h2><p>This model is an example of an electrical power generation problem (also known as a unit commitment problem). It selects an optimal set of power stations to turn on in order to satisfy anticipated power demand over a 24-hour time horizon.</p>
<p>This model is example 15 from the fifth edition of Model Building in Mathematical Programming, by H. Paul Williams on pages 270-271 and 325-326.</p>
<p>This example is of intermediate difficulty; we assume that you know Python and the Gurobi Python API and that you have some knowledge of building mathematical optimization models.</p>
<p><strong>Download the Repository</strong> <br/>
You can download the repository containing this and other examples by clicking <a href="https://github.com/Gurobi/modeling-examples/archive/master.zip">here</a>.</p>
<p><strong>Gurobi License</strong> <br/>
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an <a href="https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=3PW&amp;utm_medium=OT&amp;utm_campaign=WW-MU-EGU-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_electrical-power-generation_1_COM_EVAL_GITHUB_&amp;utm_term=electrical-power-generation-problem&amp;utm_content=C_JPM">evaluation license</a> as a <em>commercial user</em>, or download a <a href="https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=3PW&amp;utm_medium=OT&amp;utm_campaign=WW-MU-EGU-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_electrical-power-generation_1_ACADEMIC_EVAL_GITHUB_&amp;utm_term=electrical-power-generation-problem&amp;utm_content=C_JPM">free license</a> as an <em>academic user</em>.</p>
<hr/>
<h2 id="Problem-Description">Problem Description<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power_1.html#Problem-Description">¶</a></h2><p>In this problem, power generation units are grouped into three distinct types, with different characteristics for each type (power output, cost per megawatt hour, startup cost, etc.).  A unit can be on or off, with a startup cost associated with transitioning from off to on, and power output that can fall anywhere between a specified minimum and maximum value when the unit is on.  A 24-hour time horizon is divided into 5 discrete time periods, each with an expected total power demand.  The model decides which units to turn on, and when, in order to satisfy demand for each time period.  The model also captures a reserve requirement, where the selected power plants must be capable of increasing their output, while still respecting their maximum output, in order to cope with the situation where actual demand exceeds predicted demand.</p>
<p>A set of generators is available to satisfy power demand for the following day.  Anticipated demand is as follows:</p>
<table>
<thead><tr>
<th>Time Period</th>
<th>Demand (megawatts)</th>
</tr>
</thead>
<tbody>
<tr>
<td>12 pm to 6 am</td>
<td>15000</td>
</tr>
<tr>
<td>6 am to 9 am</td>
<td>30000</td>
</tr>
<tr>
<td>9 am to 3 pm</td>
<td>25000</td>
</tr>
<tr>
<td>3 pm to 6 pm</td>
<td>40000</td>
</tr>
<tr>
<td>6 pm to 12 pm</td>
<td>27000</td>
</tr>
</tbody>
</table>
<p>Generators are grouped into three types, with the following minimum and maximum output for each type (when they are on):</p>
<table>
<thead><tr>
<th>Type</th>
<th>Number available</th>
<th>Minimum output (MW)</th>
<th>Maximum output (MW)</th>
</tr>
</thead>
<tbody>
<tr>
<td>0</td>
<td>12</td>
<td>850</td>
<td>2000</td>
</tr>
<tr>
<td>1</td>
<td>10</td>
<td>1250</td>
<td>1750</td>
</tr>
<tr>
<td>2</td>
<td>5</td>
<td>1500</td>
<td>4000</td>
</tr>
</tbody>
</table>
<p>There are costs associated with using a generator: a cost per hour when the generator is on (and generating its minimum output), a cost per megawatt hour above its minimum, and a startup cost for turning a generator on:</p>
<table>
<thead><tr>
<th>Type</th>
<th>Cost per hour (when on)</th>
<th>Cost per MWh above minimum</th>
<th>Startup cost</th>
</tr>
</thead>
<tbody>
<tr>
<td>0</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-1-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1" style="width: 3.059em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.43em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-2"><span class="mi" id="MathJax-Span-3" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-4" style="font-family: MathJax_Main;">1000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1000</mn></math></span></span><script id="MathJax-Element-1" type="math/tex">\$1000</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2.00&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-2-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-5" style="width: 2.781em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.295em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.23em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-6"><span class="mi" id="MathJax-Span-7" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-8" style="font-family: MathJax_Main;">2.00</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2.00</mn></math></span></span><script id="MathJax-Element-2" type="math/tex">\$2.00</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-3-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-9" style="width: 3.059em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.43em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-10"><span class="mi" id="MathJax-Span-11" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-12" style="font-family: MathJax_Main;">2000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2000</mn></math></span></span><script id="MathJax-Element-3" type="math/tex">\$2000</script></td>
</tr>
<tr>
<td>1</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;2600&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-4-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-13" style="width: 3.059em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.43em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-14"><span class="mi" id="MathJax-Span-15" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-16" style="font-family: MathJax_Main;">2600</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>2600</mn></math></span></span><script id="MathJax-Element-4" type="math/tex">\$2600</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1.30&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-5-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-17" style="width: 2.781em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.295em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.23em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-18"><span class="mi" id="MathJax-Span-19" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-20" style="font-family: MathJax_Main;">1.30</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1.30</mn></math></span></span><script id="MathJax-Element-5" type="math/tex">\$1.30</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-6-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-21" style="width: 3.059em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.43em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-22"><span class="mi" id="MathJax-Span-23" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-24" style="font-family: MathJax_Main;">1000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1000</mn></math></span></span><script id="MathJax-Element-6" type="math/tex">\$1000</script></td>
</tr>
<tr>
<td>2</td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;3000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-7-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-25" style="width: 3.059em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.43em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-26"><span class="mi" id="MathJax-Span-27" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-28" style="font-family: MathJax_Main;">3000</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>3000</mn></math></span></span><script id="MathJax-Element-7" type="math/tex">\$3000</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;3.00&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-8-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-29" style="width: 2.781em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.295em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1002.23em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-30"><span class="mi" id="MathJax-Span-31" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-32" style="font-family: MathJax_Main;">3.00</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>3.00</mn></math></span></span><script id="MathJax-Element-8" type="math/tex">\$3.00</script></td>
<td><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;500&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-9-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-33" style="width: 2.434em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.017em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.184em, 1001.95em, 2.434em, -999.997em); top: -2.149em; left: 0em;"><span class="mrow" id="MathJax-Span-34"><span class="mi" id="MathJax-Span-35" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-36" style="font-family: MathJax_Main;">500</span></span><span style="display: inline-block; width: 0px; height: 2.156em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.163em; border-left: 0px solid; width: 0px; height: 1.171em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>500</mn></math></span></span><script id="MathJax-Element-9" type="math/tex">\$500</script></td>
</tr>
</tbody>
</table>
<p>Generators must meet predicted demand, but they must also have sufficient reserve capacity to be able to cope with the situation where actual demand exceeds predicted demand.  For this model, the set of selected generators must be able to produce as much as 115% of predicted demand.</p>
<p>Which generators should be committed to meeting anticipated demand in order to minimize total cost?</p>
<hr/>
<h2 id="Model-Formulation">Model Formulation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power_1.html#Model-Formulation">¶</a></h2><h3 id="Sets-and-Indices">Sets and Indices<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power_1.html#Sets-and-Indices">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-10-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-37" style="width: 10.896em; display: inline-block;"><span style="display: inline-block; position: relative; width: 9.051em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1008.99em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-38"><span class="mi" id="MathJax-Span-39" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-40" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-41" style="font-family: MathJax_Main; padding-left: 0.301em;">Types</span><span class="mo" id="MathJax-Span-42" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-43" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-44" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-45" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-46" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-47" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-48" style="font-family: MathJax_Main; padding-left: 0.182em;">2</span><span class="mo" id="MathJax-Span-49" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi><mo>∈</mo><mtext>Types</mtext><mo>=</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo>,</mo><mn>2</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-10" type="math/tex">t \in \text{Types}=\{0,1,2\}</script>: Types of generators.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;2&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;3&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;4&lt;/mn&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-11-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-50" style="width: 14.11em; display: inline-block;"><span style="display: inline-block; position: relative; width: 11.729em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1011.67em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-51"><span class="mi" id="MathJax-Span-52" style="font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-53" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-54" style="font-family: MathJax_Main; padding-left: 0.301em;">Periods</span><span class="mo" id="MathJax-Span-55" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-56" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mn" id="MathJax-Span-57" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-58" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-59" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-60" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-61" style="font-family: MathJax_Main; padding-left: 0.182em;">2</span><span class="mo" id="MathJax-Span-62" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-63" style="font-family: MathJax_Main; padding-left: 0.182em;">3</span><span class="mo" id="MathJax-Span-64" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-65" style="font-family: MathJax_Main; padding-left: 0.182em;">4</span><span class="mo" id="MathJax-Span-66" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi><mo>∈</mo><mtext>Periods</mtext><mo>=</mo><mo fence="false" stretchy="false">{</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo>,</mo><mn>2</mn><mo>,</mo><mn>3</mn><mo>,</mo><mn>4</mn><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-11" type="math/tex">p \in \text{Periods}=\{0,1,2,3,4\}</script>: Time periods.</p>
<h3 id="Parameters">Parameters<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power_1.html#Parameters">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;period_hours&lt;/mtext&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;N&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-12-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-67" style="width: 10.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 8.693em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1008.69em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-68"><span class="msubsup" id="MathJax-Span-69"><span style="display: inline-block; position: relative; width: 6.074em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1005.6em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-70" style="font-family: MathJax_Main;">period_hours</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.598em;"><span class="mi" id="MathJax-Span-71" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-72" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-73" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-74"><span class="mrow" id="MathJax-Span-75"><span class="mi" id="MathJax-Span-76" style="font-family: MathJax_AMS;">N</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-77" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.575em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>period_hours</mtext><mi>p</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-12" type="math/tex">\text{period_hours}_p \in \mathbb{N}^+</script>: Number of hours in each time period.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;generators&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;N&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-13-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-78" style="width: 8.932em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.443em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1007.44em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-79"><span class="msubsup" id="MathJax-Span-80"><span style="display: inline-block; position: relative; width: 4.824em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1004.47em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-81" style="font-family: MathJax_Main;">generators</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 4.527em;"><span class="mi" id="MathJax-Span-82" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-83" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-84" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-85"><span class="mrow" id="MathJax-Span-86"><span class="mi" id="MathJax-Span-87" style="font-family: MathJax_AMS;">N</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-88" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>generators</mtext><mi>t</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-13" type="math/tex">\text{generators}_t \in \mathbb{N}^+</script>: Number of generators of type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-14-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-89" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-90"><span class="mi" id="MathJax-Span-91" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-14" type="math/tex">t</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;demand&lt;/mtext&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-15-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-92" style="width: 7.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.491em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.49em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-93"><span class="msubsup" id="MathJax-Span-94"><span style="display: inline-block; position: relative; width: 3.872em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.46em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-95" style="font-family: MathJax_Main;">demand</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.455em;"><span class="mi" id="MathJax-Span-96" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-97" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-98" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-99"><span class="mrow" id="MathJax-Span-100"><span class="mi" id="MathJax-Span-101" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-102" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>demand</mtext><mi>p</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-15" type="math/tex">\text{demand}_p \in \mathbb{R}^+</script>: Total power demand for time period <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-16-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-103" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-104"><span class="mi" id="MathJax-Span-105" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-16" type="math/tex">p</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mtext&gt;start0&lt;/mtext&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;N&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-17-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-106" style="width: 6.253em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.182em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1005.18em, 2.443em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-107"><span class="mtext" id="MathJax-Span-108" style="font-family: MathJax_Main;">start0</span><span class="mo" id="MathJax-Span-109" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-110" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-111"><span class="mrow" id="MathJax-Span-112"><span class="mi" id="MathJax-Span-113" style="font-family: MathJax_AMS;">N</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-114" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.139em; border-left: 0px solid; width: 0px; height: 1.146em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mtext>start0</mtext><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-17" type="math/tex">\text{start0} \in \mathbb{N}^+</script>: Number of generators that are on at the beginning of the time horizon (and available in time period 0 without paying a startup cost).</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;min_output&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-18-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-115" style="width: 9.646em; display: inline-block;"><span style="display: inline-block; position: relative; width: 8.039em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1008.04em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-116"><span class="msubsup" id="MathJax-Span-117"><span style="display: inline-block; position: relative; width: 5.42em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1005.06em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-118" style="font-family: MathJax_Main;">min_output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.122em;"><span class="mi" id="MathJax-Span-119" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-120" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-121" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-122"><span class="mrow" id="MathJax-Span-123"><span class="mi" id="MathJax-Span-124" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-125" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>min_output</mtext><mi>t</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-18" type="math/tex">\text{min_output}_t \in \mathbb{R}^+</script>: Minimum output for generator type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-19-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-126" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-127"><span class="mi" id="MathJax-Span-128" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-19" type="math/tex">t</script> (when on).</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;max_output&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-20-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-129" style="width: 9.943em; display: inline-block;"><span style="display: inline-block; position: relative; width: 8.277em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1008.28em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-130"><span class="msubsup" id="MathJax-Span-131"><span style="display: inline-block; position: relative; width: 5.658em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1005.24em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-132" style="font-family: MathJax_Main;">max_output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.301em;"><span class="mi" id="MathJax-Span-133" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-134" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-135" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-136"><span class="mrow" id="MathJax-Span-137"><span class="mi" id="MathJax-Span-138" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-139" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>max_output</mtext><mi>t</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-20" type="math/tex">\text{max_output}_t \in \mathbb{R}^+</script>: Maximum output for generator type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-21-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-140" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-141"><span class="mi" id="MathJax-Span-142" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-21" type="math/tex">t</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;base_cost&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-22-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-143" style="width: 8.515em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.086em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1007.09em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-144"><span class="msubsup" id="MathJax-Span-145"><span style="display: inline-block; position: relative; width: 4.467em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1004.05em, 4.229em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-146" style="font-family: MathJax_Main;">base_cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 4.11em;"><span class="mi" id="MathJax-Span-147" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-148" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-149" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-150"><span class="mrow" id="MathJax-Span-151"><span class="mi" id="MathJax-Span-152" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-153" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>base_cost</mtext><mi>t</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-22" type="math/tex">\text{base_cost}_t \in \mathbb{R}^+</script>: Minimum operating cost (per hour) for a generator of type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-23-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-154" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-155"><span class="mi" id="MathJax-Span-156" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-23" type="math/tex">t</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;per_mw_cost&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-24-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-157" style="width: 10.36em; display: inline-block;"><span style="display: inline-block; position: relative; width: 8.634em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1008.63em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-158"><span class="msubsup" id="MathJax-Span-159"><span style="display: inline-block; position: relative; width: 6.015em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1005.6em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-160" style="font-family: MathJax_Main;">per_mw_cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.658em;"><span class="mi" id="MathJax-Span-161" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-162" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-163" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-164"><span class="mrow" id="MathJax-Span-165"><span class="mi" id="MathJax-Span-166" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-167" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>per_mw_cost</mtext><mi>t</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-24" type="math/tex">\text{per_mw_cost}_t \in \mathbb{R}^+</script>: Cost to generate one additional MW (per hour) for a generator of type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-25-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-168" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-169"><span class="mi" id="MathJax-Span-170" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-25" type="math/tex">t</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;startup_cost&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-26-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-171" style="width: 10.003em; display: inline-block;"><span style="display: inline-block; position: relative; width: 8.336em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1008.34em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-172"><span class="msubsup" id="MathJax-Span-173"><span style="display: inline-block; position: relative; width: 5.717em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1005.36em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-174" style="font-family: MathJax_Main;">startup_cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.42em;"><span class="mi" id="MathJax-Span-175" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-176" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-177" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-178"><span class="mrow" id="MathJax-Span-179"><span class="mi" id="MathJax-Span-180" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-181" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>startup_cost</mtext><mi>t</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-26" type="math/tex">\text{startup_cost}_t \in \mathbb{R}^+</script>: Startup cost for generator of type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-27-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-182" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-183"><span class="mi" id="MathJax-Span-184" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-27" type="math/tex">t</script>.</p>
<h3 id="Decision-Variables">Decision Variables<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power_1.html#Decision-Variables">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;ngen&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;N&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-28-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-185" style="width: 6.67em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1005.54em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-186"><span class="msubsup" id="MathJax-Span-187"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-188" style="font-family: MathJax_Main;">ngen</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-189"><span class="mrow" id="MathJax-Span-190"><span class="mi" id="MathJax-Span-191" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-192" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-193" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-194" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-195" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-196"><span class="mrow" id="MathJax-Span-197"><span class="mi" id="MathJax-Span-198" style="font-family: MathJax_AMS;">N</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-199" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.575em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>ngen</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-28" type="math/tex">\text{ngen}_{t,p} \in \mathbb{N}^+</script>: Number of generators of type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-29-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-200" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-201"><span class="mi" id="MathJax-Span-202" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-29" type="math/tex">t</script> that are on in time period <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-30-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-203" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-204"><span class="mi" id="MathJax-Span-205" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-30" type="math/tex">p</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;output&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-31-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-206" style="width: 7.741em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.432em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.43em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-207"><span class="msubsup" id="MathJax-Span-208"><span style="display: inline-block; position: relative; width: 3.812em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1002.92em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-209" style="font-family: MathJax_Main;">output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.92em;"><span class="texatom" id="MathJax-Span-210"><span class="mrow" id="MathJax-Span-211"><span class="mi" id="MathJax-Span-212" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-213" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-214" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-215" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-216" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-217"><span class="mrow" id="MathJax-Span-218"><span class="mi" id="MathJax-Span-219" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-220" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.575em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>output</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-31" type="math/tex">\text{output}_{t,p} \in \mathbb{R}^+</script>: Total power output from generators of type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-32-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-221" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-222"><span class="mi" id="MathJax-Span-223" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-32" type="math/tex">t</script> in time period <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-33-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-224" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-225"><span class="mi" id="MathJax-Span-226" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-33" type="math/tex">p</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;nstart&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;N&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-34-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-227" style="width: 7.384em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.134em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.13em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-228"><span class="msubsup" id="MathJax-Span-229"><span style="display: inline-block; position: relative; width: 3.515em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1002.56em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-230" style="font-family: MathJax_Main;">nstart</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 2.622em;"><span class="texatom" id="MathJax-Span-231"><span class="mrow" id="MathJax-Span-232"><span class="mi" id="MathJax-Span-233" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-234" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-235" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-236" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-237" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-238"><span class="mrow" id="MathJax-Span-239"><span class="mi" id="MathJax-Span-240" style="font-family: MathJax_AMS;">N</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-241" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>nstart</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-34" type="math/tex">\text{nstart}_{t,p} \in \mathbb{N}^+</script>: Number of generators of type <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-35-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-242" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-243"><span class="mi" id="MathJax-Span-244" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-35" type="math/tex">t</script> to start in time period <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-36-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-245" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-246"><span class="mi" id="MathJax-Span-247" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-36" type="math/tex">p</script>.</p>
<h3 id="Objective-Function">Objective Function<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power_1.html#Objective-Function">¶</a></h3><ul>
<li><strong>Cost</strong>: Minimize the cost (in USD) to satisfy the predicted electricity demand.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtext&gt;Minimize&lt;/mtext&gt;&lt;mspace width="1em" /&gt;&lt;msub&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;o&lt;/mi&gt;&lt;mi&gt;n&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;e&lt;/mi&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;u&lt;/mi&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/math&gt;' id="MathJax-Element-37-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-248" style="width: 17.503em; display: inline-block;"><span style="display: inline-block; position: relative; width: 14.586em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1014.59em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-249"><span class="mtext" id="MathJax-Span-250" style="font-family: MathJax_Main;">Minimize</span><span class="mspace" id="MathJax-Span-251" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="msubsup" id="MathJax-Span-252"><span style="display: inline-block; position: relative; width: 1.551em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-253" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.658em;"><span class="texatom" id="MathJax-Span-254"><span class="mrow" id="MathJax-Span-255"><span class="mi" id="MathJax-Span-256" style="font-size: 70.7%; font-family: MathJax_Math-italic;">o</span><span class="mi" id="MathJax-Span-257" style="font-size: 70.7%; font-family: MathJax_Math-italic;">n</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-258" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-259" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.443em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-260" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.658em;"><span class="texatom" id="MathJax-Span-261"><span class="mrow" id="MathJax-Span-262"><span class="mi" id="MathJax-Span-263" style="font-size: 70.7%; font-family: MathJax_Math-italic;">e</span><span class="mi" id="MathJax-Span-264" style="font-size: 70.7%; font-family: MathJax_Math-italic;">x</span><span class="mi" id="MathJax-Span-265" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-266" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-267" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-268" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-269" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.039em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-270" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.658em;"><span class="texatom" id="MathJax-Span-271"><span class="mrow" id="MathJax-Span-272"><span class="mi" id="MathJax-Span-273" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mi" id="MathJax-Span-274" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-275" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span class="mi" id="MathJax-Span-276" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-277" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-278" style="font-size: 70.7%; font-family: MathJax_Math-italic;">u</span><span class="mi" id="MathJax-Span-279" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtext>Minimize</mtext><mspace width="1em"></mspace><msub><mi>Z</mi><mrow class="MJX-TeXAtom-ORD"><mi>o</mi><mi>n</mi></mrow></msub><mo>+</mo><msub><mi>Z</mi><mrow class="MJX-TeXAtom-ORD"><mi>e</mi><mi>x</mi><mi>t</mi><mi>r</mi><mi>a</mi></mrow></msub><mo>+</mo><msub><mi>Z</mi><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mi>t</mi><mi>a</mi><mi>r</mi><mi>t</mi><mi>u</mi><mi>p</mi></mrow></msub></math></span></span></div><script id="MathJax-Element-37" type="math/tex; mode=display">\begin{equation}
\text{Minimize} \quad Z_{on} + Z_{extra} + Z_{startup}
\end{equation}</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;o&lt;/mi&gt;&lt;mi&gt;n&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;base_cost&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;ngen&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/math&gt;' id="MathJax-Element-38-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-280" style="width: 21.67em; display: inline-block;"><span style="display: inline-block; position: relative; width: 18.039em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1018.04em, 3.812em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-281"><span class="msubsup" id="MathJax-Span-282"><span style="display: inline-block; position: relative; width: 1.551em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-283" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.658em;"><span class="texatom" id="MathJax-Span-284"><span class="mrow" id="MathJax-Span-285"><span class="mi" id="MathJax-Span-286" style="font-size: 70.7%; font-family: MathJax_Math-italic;">o</span><span class="mi" id="MathJax-Span-287" style="font-size: 70.7%; font-family: MathJax_Math-italic;">n</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-288" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-289" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 6.551em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 2.562em;"><span class="mo" id="MathJax-Span-290" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.277em, 1006.55em, 4.467em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-291"><span class="mrow" id="MathJax-Span-292"><span class="mo" id="MathJax-Span-293" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-294" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-295" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-296" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-297" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-298" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-299" style="font-size: 70.7%; font-family: MathJax_Main;">Types</span><span class="mo" id="MathJax-Span-300" style="font-size: 70.7%; font-family: MathJax_Main;">×</span><span class="mtext" id="MathJax-Span-301" style="font-size: 70.7%; font-family: MathJax_Main;">Periods</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-302" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-303"><span class="msubsup" id="MathJax-Span-304"><span style="display: inline-block; position: relative; width: 4.467em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1004.05em, 4.229em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-305" style="font-family: MathJax_Main;">base_cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 4.11em;"><span class="mi" id="MathJax-Span-306" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-307" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-308" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-309" style="font-family: MathJax_Main;">ngen</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-310"><span class="mrow" id="MathJax-Span-311"><span class="mi" id="MathJax-Span-312" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-313" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-314" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.782em; border-left: 0px solid; width: 0px; height: 3.004em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>Z</mi><mrow class="MJX-TeXAtom-ORD"><mi>o</mi><mi>n</mi></mrow></msub><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Types</mtext><mo>×</mo><mtext>Periods</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>base_cost</mtext><mi>t</mi></msub><mo>∗</mo><msub><mtext>ngen</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow></math></span></span></div><script id="MathJax-Element-38" type="math/tex; mode=display">\begin{equation}
Z_{on} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{base_cost}_t*\text{ngen}_{t,p}}
\end{equation}</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;e&lt;/mi&gt;&lt;mi&gt;x&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;per_mw_cost&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;output&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;min_load&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-39-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-315" style="width: 33.336em; display: inline-block;"><span style="display: inline-block; position: relative; width: 27.741em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1027.62em, 3.812em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-316"><span class="msubsup" id="MathJax-Span-317"><span style="display: inline-block; position: relative; width: 2.443em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-318" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.658em;"><span class="texatom" id="MathJax-Span-319"><span class="mrow" id="MathJax-Span-320"><span class="mi" id="MathJax-Span-321" style="font-size: 70.7%; font-family: MathJax_Math-italic;">e</span><span class="mi" id="MathJax-Span-322" style="font-size: 70.7%; font-family: MathJax_Math-italic;">x</span><span class="mi" id="MathJax-Span-323" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-324" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-325" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-326" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-327" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 6.551em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 2.562em;"><span class="mo" id="MathJax-Span-328" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.277em, 1006.55em, 4.467em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-329"><span class="mrow" id="MathJax-Span-330"><span class="mo" id="MathJax-Span-331" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-332" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-333" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-334" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-335" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-336" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-337" style="font-size: 70.7%; font-family: MathJax_Main;">Types</span><span class="mo" id="MathJax-Span-338" style="font-size: 70.7%; font-family: MathJax_Main;">×</span><span class="mtext" id="MathJax-Span-339" style="font-size: 70.7%; font-family: MathJax_Main;">Periods</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-340" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-341"><span class="msubsup" id="MathJax-Span-342"><span style="display: inline-block; position: relative; width: 6.015em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1005.6em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-343" style="font-family: MathJax_Main;">per_mw_cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.658em;"><span class="mi" id="MathJax-Span-344" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-345" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="mo" id="MathJax-Span-346" style="font-family: MathJax_Main; padding-left: 0.241em;">(</span><span class="msubsup" id="MathJax-Span-347"><span style="display: inline-block; position: relative; width: 3.812em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1002.92em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-348" style="font-family: MathJax_Main;">output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.92em;"><span class="texatom" id="MathJax-Span-349"><span class="mrow" id="MathJax-Span-350"><span class="mi" id="MathJax-Span-351" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-352" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-353" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-354" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="msubsup" id="MathJax-Span-355" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 4.348em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.99em, 4.229em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-356" style="font-family: MathJax_Main;">min_load</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.991em;"><span class="mi" id="MathJax-Span-357" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-358" style="font-family: MathJax_Main;">)</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.782em; border-left: 0px solid; width: 0px; height: 3.004em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>Z</mi><mrow class="MJX-TeXAtom-ORD"><mi>e</mi><mi>x</mi><mi>t</mi><mi>r</mi><mi>a</mi></mrow></msub><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Types</mtext><mo>×</mo><mtext>Periods</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>per_mw_cost</mtext><mi>t</mi></msub><mo>∗</mo><mo stretchy="false">(</mo><msub><mtext>output</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>−</mo><msub><mtext>min_load</mtext><mi>t</mi></msub></mrow><mo stretchy="false">)</mo></math></span></span></div><script id="MathJax-Element-39" type="math/tex; mode=display">\begin{equation}
Z_{extra} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{per_mw_cost}_t*(\text{output}_{t,p} - \text{min_load}_t})
\end{equation}</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;u&lt;/mi&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;startup_cost&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;nstart&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;/math&gt;' id="MathJax-Element-40-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-359" style="width: 25.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 21.372em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1021.37em, 3.812em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-360"><span class="msubsup" id="MathJax-Span-361"><span style="display: inline-block; position: relative; width: 3.039em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mi" id="MathJax-Span-362" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 0.658em;"><span class="texatom" id="MathJax-Span-363"><span class="mrow" id="MathJax-Span-364"><span class="mi" id="MathJax-Span-365" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mi" id="MathJax-Span-366" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-367" style="font-size: 70.7%; font-family: MathJax_Math-italic;">a</span><span class="mi" id="MathJax-Span-368" style="font-size: 70.7%; font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-369" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-370" style="font-size: 70.7%; font-family: MathJax_Math-italic;">u</span><span class="mi" id="MathJax-Span-371" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-372" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-373" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 6.551em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 2.562em;"><span class="mo" id="MathJax-Span-374" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.277em, 1006.55em, 4.467em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-375"><span class="mrow" id="MathJax-Span-376"><span class="mo" id="MathJax-Span-377" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-378" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-379" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-380" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-381" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-382" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-383" style="font-size: 70.7%; font-family: MathJax_Main;">Types</span><span class="mo" id="MathJax-Span-384" style="font-size: 70.7%; font-family: MathJax_Main;">×</span><span class="mtext" id="MathJax-Span-385" style="font-size: 70.7%; font-family: MathJax_Main;">Periods</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-386" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-387"><span class="msubsup" id="MathJax-Span-388"><span style="display: inline-block; position: relative; width: 5.717em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1005.36em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-389" style="font-family: MathJax_Main;">startup_cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.42em;"><span class="mi" id="MathJax-Span-390" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-391" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-392" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.515em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1002.56em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-393" style="font-family: MathJax_Main;">nstart</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 2.622em;"><span class="texatom" id="MathJax-Span-394"><span class="mrow" id="MathJax-Span-395"><span class="mi" id="MathJax-Span-396" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-397" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-398" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.782em; border-left: 0px solid; width: 0px; height: 3.004em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mi>Z</mi><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mi>t</mi><mi>a</mi><mi>r</mi><mi>t</mi><mi>u</mi><mi>p</mi></mrow></msub><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Types</mtext><mo>×</mo><mtext>Periods</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>startup_cost</mtext><mi>t</mi></msub><mo>∗</mo><msub><mtext>nstart</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow></math></span></span></div><script id="MathJax-Element-40" type="math/tex; mode=display">\begin{equation}
Z_{startup} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{startup_cost}_t*\text{nstart}_{t,p}}
\end{equation}</script><h3 id="Constraints">Constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power_1.html#Constraints">¶</a></h3><ul>
<li><strong>Available generators</strong>: Number of generators used must be less than or equal to the number available.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mtext&gt;ngen&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;generators&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-41-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-399" style="width: 25.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 21.193em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1021.13em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-400"><span class="msubsup" id="MathJax-Span-401"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-402" style="font-family: MathJax_Main;">ngen</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-403"><span class="mrow" id="MathJax-Span-404"><span class="mi" id="MathJax-Span-405" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-406" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-407" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-408" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-409" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 4.824em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1004.47em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-410" style="font-family: MathJax_Main;">generators</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 4.527em;"><span class="texatom" id="MathJax-Span-411"><span class="mrow" id="MathJax-Span-412"><span class="mi" id="MathJax-Span-413" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-414" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-415" style="font-family: MathJax_Main;">∀</span><span class="mo" id="MathJax-Span-416" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-417" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-418" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-419" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">p</span><span class="mo" id="MathJax-Span-420" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-421" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-422" style="font-family: MathJax_Main; padding-left: 0.301em;">Types</span><span class="mo" id="MathJax-Span-423" style="font-family: MathJax_Main; padding-left: 0.241em;">×</span><span class="mtext" id="MathJax-Span-424" style="font-family: MathJax_Main; padding-left: 0.241em;">Periods</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>ngen</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>≤</mo><msub><mtext>generators</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Types</mtext><mo>×</mo><mtext>Periods</mtext></math></span></span></div><script id="MathJax-Element-41" type="math/tex; mode=display">\begin{equation}
\text{ngen}_{t,p} \leq \text{generators}_{t} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}</script><ul>
<li><strong>Demand</strong>: Total power generated across all generator types must meet anticipated demand for each time period <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-42-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-425" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.48em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-426"><span class="mi" id="MathJax-Span-427" style="font-family: MathJax_Math-italic;">p</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>p</mi></math></span></span><script id="MathJax-Element-42" type="math/tex">p</script>.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;output&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;demand&lt;/mtext&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-43-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-428" style="width: 22.146em; display: inline-block;"><span style="display: inline-block; position: relative; width: 18.455em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1018.4em, 3.753em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-429"><span class="munderover" id="MathJax-Span-430"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.598em;"><span class="mo" id="MathJax-Span-431" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1002.56em, 4.408em, -999.997em); top: -2.914em; left: 0em;"><span class="texatom" id="MathJax-Span-432"><span class="mrow" id="MathJax-Span-433"><span class="mi" id="MathJax-Span-434" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-435" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-436" style="font-size: 70.7%; font-family: MathJax_Main;">Types</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-437" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-438"><span class="msubsup" id="MathJax-Span-439"><span style="display: inline-block; position: relative; width: 3.812em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1002.92em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-440" style="font-family: MathJax_Main;">output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.92em;"><span class="texatom" id="MathJax-Span-441"><span class="mrow" id="MathJax-Span-442"><span class="mi" id="MathJax-Span-443" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-444" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-445" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-446" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="msubsup" id="MathJax-Span-447" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 3.872em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.46em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-448" style="font-family: MathJax_Main;">demand</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.455em;"><span class="mi" id="MathJax-Span-449" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-450" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-451" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-452" style="font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-453" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-454" style="font-family: MathJax_Main; padding-left: 0.301em;">Periods</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.711em; border-left: 0px solid; width: 0px; height: 2.861em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mtext>Types</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>output</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow><mo>≥</mo><msub><mtext>demand</mtext><mi>p</mi></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>p</mi><mo>∈</mo><mtext>Periods</mtext></math></span></span></div><script id="MathJax-Element-43" type="math/tex; mode=display">\begin{equation}
\sum_{t \in \text{Types}}{\text{output}_{t,p}} \geq \text{demand}_p \quad \forall p \in \text{Periods}
\end{equation}</script><ul>
<li><strong>Min/max generation</strong>: Power generation must respect generator min/max values.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mtext&gt;output&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;min_output&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;ngen&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-44-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-455" style="width: 31.967em; display: inline-block;"><span style="display: inline-block; position: relative; width: 26.61em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1026.55em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-456"><span class="msubsup" id="MathJax-Span-457"><span style="display: inline-block; position: relative; width: 3.812em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1002.92em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-458" style="font-family: MathJax_Main;">output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.92em;"><span class="texatom" id="MathJax-Span-459"><span class="mrow" id="MathJax-Span-460"><span class="mi" id="MathJax-Span-461" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-462" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-463" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-464" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="msubsup" id="MathJax-Span-465" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 5.42em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1005.06em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-466" style="font-family: MathJax_Main;">min_output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.122em;"><span class="mi" id="MathJax-Span-467" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-468" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-469" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-470" style="font-family: MathJax_Main;">ngen</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-471"><span class="mrow" id="MathJax-Span-472"><span class="mi" id="MathJax-Span-473" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-474" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-475" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-476" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-477" style="font-family: MathJax_Main;">∀</span><span class="mo" id="MathJax-Span-478" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-479" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-480" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-481" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">p</span><span class="mo" id="MathJax-Span-482" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-483" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-484" style="font-family: MathJax_Main; padding-left: 0.301em;">Types</span><span class="mo" id="MathJax-Span-485" style="font-family: MathJax_Main; padding-left: 0.241em;">×</span><span class="mtext" id="MathJax-Span-486" style="font-family: MathJax_Main; padding-left: 0.241em;">Periods</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>output</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>≥</mo><msub><mtext>min_output</mtext><mi>t</mi></msub><mo>∗</mo><msub><mtext>ngen</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Types</mtext><mo>×</mo><mtext>Periods</mtext></math></span></span></div><script id="MathJax-Element-44" type="math/tex; mode=display">\begin{equation}
\text{output}_{t,p} \geq \text{min_output}_t*\text{ngen}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mtext&gt;output&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;max_output&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;ngen&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-45-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-487" style="width: 32.265em; display: inline-block;"><span style="display: inline-block; position: relative; width: 26.848em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1026.79em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-488"><span class="msubsup" id="MathJax-Span-489"><span style="display: inline-block; position: relative; width: 3.812em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1002.92em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-490" style="font-family: MathJax_Main;">output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.92em;"><span class="texatom" id="MathJax-Span-491"><span class="mrow" id="MathJax-Span-492"><span class="mi" id="MathJax-Span-493" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-494" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-495" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-496" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-497" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 5.658em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1005.24em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-498" style="font-family: MathJax_Main;">max_output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.301em;"><span class="mi" id="MathJax-Span-499" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-500" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-501" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-502" style="font-family: MathJax_Main;">ngen</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-503"><span class="mrow" id="MathJax-Span-504"><span class="mi" id="MathJax-Span-505" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-506" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-507" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-508" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-509" style="font-family: MathJax_Main;">∀</span><span class="mo" id="MathJax-Span-510" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-511" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-512" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-513" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">p</span><span class="mo" id="MathJax-Span-514" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-515" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-516" style="font-family: MathJax_Main; padding-left: 0.301em;">Types</span><span class="mo" id="MathJax-Span-517" style="font-family: MathJax_Main; padding-left: 0.241em;">×</span><span class="mtext" id="MathJax-Span-518" style="font-family: MathJax_Main; padding-left: 0.241em;">Periods</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>output</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>≤</mo><msub><mtext>max_output</mtext><mi>t</mi></msub><mo>∗</mo><msub><mtext>ngen</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Types</mtext><mo>×</mo><mtext>Periods</mtext></math></span></span></div><script id="MathJax-Element-45" type="math/tex; mode=display">\begin{equation}
\text{output}_{t,p} \leq \text{max_output}_t*\text{ngen}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}</script><ul>
<li><strong>Reserve</strong>: Selected generators must be able to satisfy demand that is as much as 15% above the prediction.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;max_output&lt;/mtext&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;ngen&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2265;&lt;/mo&gt;&lt;mn&gt;1.15&lt;/mn&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;demand&lt;/mtext&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-46-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-519" style="width: 32.384em; display: inline-block;"><span style="display: inline-block; position: relative; width: 26.967em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1026.91em, 3.753em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-520"><span class="munderover" id="MathJax-Span-521"><span style="display: inline-block; position: relative; width: 2.622em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.598em;"><span class="mo" id="MathJax-Span-522" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1002.56em, 4.408em, -999.997em); top: -2.914em; left: 0em;"><span class="texatom" id="MathJax-Span-523"><span class="mrow" id="MathJax-Span-524"><span class="mi" id="MathJax-Span-525" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-526" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-527" style="font-size: 70.7%; font-family: MathJax_Main;">Types</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-528" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-529"><span class="msubsup" id="MathJax-Span-530"><span style="display: inline-block; position: relative; width: 5.658em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1005.24em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-531" style="font-family: MathJax_Main;">max_output</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 5.301em;"><span class="mi" id="MathJax-Span-532" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-533" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-534" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-535" style="font-family: MathJax_Main;">ngen</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-536"><span class="mrow" id="MathJax-Span-537"><span class="mi" id="MathJax-Span-538" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-539" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-540" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-541" style="font-family: MathJax_Main; padding-left: 0.301em;">≥</span><span class="mn" id="MathJax-Span-542" style="font-family: MathJax_Main; padding-left: 0.301em;">1.15</span><span class="mo" id="MathJax-Span-543" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-544" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 3.872em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.46em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-545" style="font-family: MathJax_Main;">demand</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.455em;"><span class="mi" id="MathJax-Span-546" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-547" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-548" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-549" style="font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-550" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-551" style="font-family: MathJax_Main; padding-left: 0.301em;">Periods</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.711em; border-left: 0px solid; width: 0px; height: 2.861em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mtext>Types</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>max_output</mtext><mi>t</mi></msub><mo>∗</mo><msub><mtext>ngen</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub></mrow><mo>≥</mo><mn>1.15</mn><mo>∗</mo><msub><mtext>demand</mtext><mi>p</mi></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>p</mi><mo>∈</mo><mtext>Periods</mtext></math></span></span></div><script id="MathJax-Element-46" type="math/tex; mode=display">\begin{equation}
\sum_{t \in \text{Types}}{\text{max_output}_t*\text{ngen}_{t,p}} \geq 1.15 * \text{demand}_p \quad \forall p \in \text{Periods}
\end{equation}</script><ul>
<li><strong>Startup</strong>: Establish relationship between number of active generators and number of startups (use <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mi&gt;a&lt;/mi&gt;&lt;mi&gt;r&lt;/mi&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-47-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-552" style="width: 3.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1002.62em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-553"><span class="mi" id="MathJax-Span-554" style="font-family: MathJax_Math-italic;">s</span><span class="mi" id="MathJax-Span-555" style="font-family: MathJax_Math-italic;">t</span><span class="mi" id="MathJax-Span-556" style="font-family: MathJax_Math-italic;">a</span><span class="mi" id="MathJax-Span-557" style="font-family: MathJax_Math-italic;">r</span><span class="mi" id="MathJax-Span-558" style="font-family: MathJax_Math-italic;">t</span><span class="mn" id="MathJax-Span-559" style="font-family: MathJax_Main;">0</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi><mi>t</mi><mi>a</mi><mi>r</mi><mi>t</mi><mn>0</mn></math></span></span><script id="MathJax-Element-47" type="math/tex">start0</script> for period before the time horizon starts)</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mtext&gt;ngen&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;ngen&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;startup&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;p&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Types&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Periods&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-48-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-560" style="width: 30.717em; display: inline-block;"><span style="display: inline-block; position: relative; width: 25.598em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1025.54em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-561"><span class="msubsup" id="MathJax-Span-562"><span style="display: inline-block; position: relative; width: 2.92em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-563" style="font-family: MathJax_Main;">ngen</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-564"><span class="mrow" id="MathJax-Span-565"><span class="mi" id="MathJax-Span-566" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-567" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-568" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-569" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-570" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 3.872em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-571" style="font-family: MathJax_Main;">ngen</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-572"><span class="mrow" id="MathJax-Span-573"><span class="mi" id="MathJax-Span-574" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-575" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-576" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span><span class="mo" id="MathJax-Span-577" style="font-size: 70.7%; font-family: MathJax_Main;">−</span><span class="mn" id="MathJax-Span-578" style="font-size: 70.7%; font-family: MathJax_Main;">1</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-579" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="msubsup" id="MathJax-Span-580" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 4.051em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1003.16em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-581" style="font-family: MathJax_Main;">startup</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.158em;"><span class="texatom" id="MathJax-Span-582"><span class="mrow" id="MathJax-Span-583"><span class="mi" id="MathJax-Span-584" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-585" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-586" style="font-size: 70.7%; font-family: MathJax_Math-italic;">p</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-587" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-588" style="font-family: MathJax_Main;">∀</span><span class="mo" id="MathJax-Span-589" style="font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-590" style="font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-591" style="font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-592" style="font-family: MathJax_Math-italic; padding-left: 0.182em;">p</span><span class="mo" id="MathJax-Span-593" style="font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-594" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-595" style="font-family: MathJax_Main; padding-left: 0.301em;">Types</span><span class="mo" id="MathJax-Span-596" style="font-family: MathJax_Main; padding-left: 0.241em;">×</span><span class="mtext" id="MathJax-Span-597" style="font-family: MathJax_Main; padding-left: 0.241em;">Periods</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>ngen</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mo>≤</mo><msub><mtext>ngen</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi><mo>−</mo><mn>1</mn></mrow></msub><mo>+</mo><msub><mtext>startup</mtext><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>,</mo><mi>p</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mo stretchy="false">(</mo><mi>t</mi><mo>,</mo><mi>p</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Types</mtext><mo>×</mo><mtext>Periods</mtext></math></span></span></div><script id="MathJax-Element-48" type="math/tex; mode=display">\begin{equation}
\text{ngen}_{t,p} \leq \text{ngen}_{t,p-1} + \text{startup}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}</script><hr/>
<h2 id="Python-Implementation">Python Implementation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power_1.html#Python-Implementation">¶</a></h2><p>We import the Gurobi Python Module and other Python libraries.</p>


In [ ]:

import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0




<h2 id="Input-Data">Input Data<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power_1.html#Input-Data">¶</a></h2><p>We define all the input data of the model.</p>


In [ ]:

# Parameters

ntypes = 3
nperiods = 5
maxstart0 = 5

generators = [12, 10, 5]
period_hours = [6, 3, 6, 3, 6]
demand = [15000, 30000, 25000, 40000, 27000]
min_load = [850, 1250, 1500]
max_load = [2000, 1750, 4000]
base_cost = [1000, 2600, 3000]
per_mw_cost = [2, 1.3, 3]
startup_cost = [2000, 1000, 500]




<h2 id="Model-Deployment">Model Deployment<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power_1.html#Model-Deployment">¶</a></h2><p>We create a model and the variables. For each time period, we have: an integer decision variable to capture the number of active generators of each type (ngen), an integer variable to capture the number of generators of each type we must start (nstart), and a continuous decision variable to capture the total power output for each generator type (output).</p>


In [ ]:

model = gp.Model('PowerGeneration')
ngen = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="ngen")
nstart = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="nstart")
output = model.addVars(ntypes, nperiods, vtype=GRB.CONTINUOUS, name="genoutput")




<p>Next we insert the constraints:</p>
<p>The number of active generators can't exceed the number of generators.</p>


In [ ]:

# Generator count

numgen = model.addConstrs(ngen[type, period] <= generators[type]
                         for type in range(ntypes) for period in range(nperiods))




<p>Total power output for a generator type depends on the number of generators of that type that are active.</p>


In [ ]:

# Respect minimum and maximum output per generator type

min_output = model.addConstrs((output[type, period] >= min_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))

max_output = model.addConstrs((output[type, period] <= max_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))




<p>Total output for each time period must meet predicted demand.</p>


In [ ]:

# Meet demand

meet_demand = model.addConstrs(gp.quicksum(output[type, period] for type in range(ntypes)) >= demand[period]
                               for period in range(nperiods))




<p>Selected generators must be able to cope with an excess of demand.</p>


In [ ]:

# Provide sufficient reserve capacity

reserve = model.addConstrs(gp.quicksum(max_load[type]*ngen[type, period] for type in range(ntypes)) >= 1.15*demand[period]
                    for period in range(nperiods))




<p>Connect the decision variables that capture active generators with the decision variables that count startups.</p>


In [ ]:

# Startup constraint

startup0 = model.addConstrs((ngen[type,0] <= maxstart0 + nstart[type,0])
                            for type in range(ntypes))

startup = model.addConstrs((ngen[type,period] <= ngen[type,period-1] + nstart[type,period])
                           for type in range(ntypes) for period in range(1,nperiods))




<p>Objective: minimize total cost.  Cost consists of three components: the cost for running active generation units, the cost to generate power beyond the minimum for each unit, and the cost to start up generation units.</p>


In [ ]:

# Objective: minimize total cost

active = gp.quicksum(base_cost[type]*period_hours[period]*ngen[type,period]
                    for type in range(ntypes) for period in range(nperiods))

per_mw = gp.quicksum(per_mw_cost[type]*period_hours[period]*(output[type,period] - min_load[type]*ngen[type,period])
                       for type in range(ntypes) for period in range(nperiods))

startup_obj = gp.quicksum(startup_cost[type]*nstart[type,period]
                         for type in range(ntypes) for period in range(nperiods))

model.setObjective(active + per_mw + startup_obj)




<p>Next, we start the optimization and Gurobi finds the optimal solution.</p>


In [ ]:

model.write('junk.lp')
model.optimize()




<hr/>
<h2 id="Analysis">Analysis<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power_1.html#Analysis">¶</a></h2><p>The anticipated demand for electricity over the 24-hour time window can be met for a total cost of <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;002&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;540&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-49-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-598" style="width: 5.955em; display: inline-block;"><span style="display: inline-block; position: relative; width: 4.943em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1004.88em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-599"><span class="mi" id="MathJax-Span-600" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-601" style="font-family: MathJax_Main;">1</span><span class="mo" id="MathJax-Span-602" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-603" style="font-family: MathJax_Main; padding-left: 0.182em;">002</span><span class="mo" id="MathJax-Span-604" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-605" style="font-family: MathJax_Main; padding-left: 0.182em;">540</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>1</mn><mo>,</mo><mn>002</mn><mo>,</mo><mn>540</mn></math></span></span><script id="MathJax-Element-49" type="math/tex">\$1,002,540</script>. The detailed plan for each time period is as follows.</p>
<h3 id="Unit-Commitments">Unit Commitments<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power_1.html#Unit-Commitments">¶</a></h3><p>The following table shows the number of generators of each type that are active in each time period in the optimal solution:</p>


In [ ]:

rows = ["Type" + str(t) for t in range(ntypes)]
units = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for t in range(ntypes):
    for p in range(nperiods):
        units.loc["Type"+str(t), p] = ngen[t,p].x
units




<p>The following shows the number of generators of each type that must be started in each time period to achieve this plan (recall that the model assumes that 5 are available at the beginning of the time horizon):</p>


In [ ]:

startups = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for t in range(ntypes):
    for p in range(nperiods):
        startups.loc["Type"+str(t), p] = int(nstart[t,p].x)
startups




<hr/>
<h2 id="References">References<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/electrical_power_generation_1_2/electrical_power_1.html#References">¶</a></h2><p>H. Paul Williams, Model Building in Mathematical Programming, fifth edition.</p>
<p>Copyright © 2020 Gurobi Optimization, LLC</p>
